In [ ]:
#### import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib
from sobol_lib import *
from mpl_toolkits.mplot3d import Axes3D
from gen_mat import *
from block_rect_maxvol import *
import re
import os
from matplotlib import cm
from mva_test import *
import itertools
from ipywidgets import interactive, interact, widgets
from sympy import *
from exp_proccess import *
from exp_setup import *
from scipy.special import comb
import pathlib

%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.widgets import Slider
from maxvolpy.maxvol import rect_maxvol, maxvol
from numba import jit, njit
os.environ['OMP_NUM_THREADS'] = '6'
print (os.environ['OMP_NUM_THREADS'])

In [ ]:
num_expr = 1 # Adding this num to output fn for distinguish between experiments  (not used for now)
nder = 4 # Dimension
ToLoadSet = False # whether to load settigns form file

cut_radiuses = [0.005]
inital_points_distribs = ['LHS']

if ToLoadSet:
    from sandbox_bm_approx_settings import *
else:
    dir_str = './cr_talk'
    #dir_str = os.path.join(os.environ['HOME'], 'work/res/bm-big/')
    num_points_for_big_matrix = 6000 # number of points for big matrix
    max_row =  18                   # Maximum number of points taken in experiments
    max_expansion = 7         # number of columns in big matrix in (nder+1) units
    min_expansion = 3        # minimal number of columns in experiments in (nder+1) units
    

    n_test = 30000    # points on test grid (for calculating error on final step)
    poly = cheb       # used polinomials

    domain_type = None    #possible types: 'blob', 'ellipse', 'plane', 'rhombus', 'circle'
    
    
#####
dir_pdf = os.path.join(dir_str, "pdf")
try:
    os.makedirs(dir_pdf)
except:
    pass



ToTakePointsFromFile = False # (not used for now)

# ---------------------------------
p_size = (nder+1)*max_row #number of rows in big matrix

### generating test points
points_test = complex_area_pnts_gen(n_test, nder, distrib='LHS', mod = domain_type)

### Testing the new most general environment

In [ ]:
config = Config()
cur_pos = str(pathlib.Path().parent.absolute())
config.design_cardinality = 150
design_pts = complex_area_pnts_gen(config.design_cardinality, 6, distrib='LHS', mod = None)

In [ ]:
config.min_expansion = 4
config.max_expansion = 6
config.expansion_set = None
config.design_dimension = 1
config.poly = poly_power
config.n_iter = 2500
config.delta_n = 40

config.design_space = design_pts[:,:config.design_dimension]

add_str = '-'.join([str(i) for i in [config.max_expansion, "_lebesgue_dim{}".format(config.design_dimension)]])
dir_str = cur_pos + '/domain_exp_' + add_str


worker = Experiment(config, dir_str)
worker.run()

In [ ]:
# np.savez(dir_str, x=design_pts)
x2 = design_pts[:,:5]
np.savez(dir_str + 'domain_exp_127-_lebesgue_dim5/domain_dim=5.npz', x = x2)

### Experimental data processing

In [ ]:
### Data extraction
dir_str = './'
ndim_input = 4
exp_folder = "domain_exp_70-_lebesgue_dim{}/".format(ndim_input)
dir_points = os.path.join(dir_str, exp_folder)
domain_type= None
block_approx = True
if block_approx:
    calibrate = ndim_input + 1
else:
    calibrate = 1
nder = ndim_input - 1
points_distrib = 'LHS'

design_space = "domain_dim={}".format(ndim_input)
test_space = "test_" + design_space
std_design_name = "designs_dim={}".format(ndim_input)
add_name = ""
add_name2 = "_wise_expand_sd"
add_name3 = "_wise_expand_wd"
add_name4 = ""
calc_design = std_design_name + add_name4 + ".txt"
# extend_design = "designs_dim={}_expnd.txt".format(nder+1)
taken_points = np.load(dir_points + design_space + ".npz")
taken_test = np.load(dir_points + test_space + ".npz")
x = taken_points['x']
points_test = taken_test['x']
#fn = dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt"

In [ ]:
### .txt processing
N_col, p_indices,iter_n,wts = file_extraction(dir_points + calc_design)

In [ ]:
ex1 = Experiment(dir_points + design_space, dir_points + std_design_name + add_name + ".txt", ndim_input, 1)

In [ ]:
ex2 = Experiment(dir_points + design_space, dir_points + std_design_name + add_name2 + ".txt", ndim_input, 1)
ex3 = Experiment(dir_points + design_space, dir_points + std_design_name + add_name3 + ".txt", ndim_input, 1)

In [ ]:
ex4 = Experiment(dir_points + design_space, dir_points + std_design_name + add_name4 + ".txt", ndim_input, 1)

In [ ]:
ex3.name = r'$|$supp $\xi|$={}'.format(len(ex1.p_indices[0]))

In [ ]:
ex3.name = r'$|$supp $\xi|$ fixed'

In [ ]:
ex2.name = r'Truncated optimal $\xi$, $|$supp $\xi|$={}'.format(len(ex2.p_indices[0])) #"Equal # and points themselves, diff expansion"
ex3.name = r'Truncated optimal $\xi$, $|$supp $\xi|$={}'.format(len(ex3.p_indices[0]))
# ex3.name = ex1.name
#ex4.name = r'Optimal $\xi$'

In [ ]:
ex2 = deepcopy(ex1)
ex2.name = "expanded to square"
ex2.change_expans("up_to_points")

In [ ]:
ex4.get_k_points(5, 'equal_to_columns')
ex4.update_cardinalities()
print(ex4.cardinalities, ex4.expans)

In [ ]:
ex2.name = r'Optimal $\tilde{\xi}$, $U(\tilde{\xi}) \in \mathbb{R}^{l \times k}$'#r'Truncated optimal $\tilde{\xi}$, $U(\tilde{\xi}) \in \mathbb{R}^{k \times k}$'
ex1.name = r'Optimal $\xi$, $U(\xi) \in \mathbb{R}^{k \times p}$'

In [ ]:
ex4.name = r'Truncated optimal $\tilde{\xi}$'
ex1.name = r'Optimal $\xi$'

In [ ]:
def plot_card_p(ex, description, marker_set=None):
    fig, ax = plt.subplots()
    ax.plot(ex.expans, ex.cardinalities, 'go', markersize=1, label=r'|supp $\xi(p)$|')
    if marker_set is None:
        pass
    else:
        ax.plot(x[marker_set], variance_set[marker_set], 'bo', markersize=3, label='Optimal')
    ax.set_xlim(0, ex.expans[-1]+5)
    ax.xaxis.set_major_locator(plt.MultipleLocator(10))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    #plt.axis('scaled')
    plt.yscale('log')
    plt.xlabel("Number of expansion terms, "+r'$p$')
    ax.legend()
    plt.grid()
    fig.savefig(description+'.pdf', bbox_inches='tight')
    plt.close()


In [ ]:
plot_card_p(ex1, dir_points+'card_ndim{}'.format(ndim_input))

In [ ]:
def increase_support(p_indices, percent):
    for i, sup_set in enumerate(p_indices):
        k = len(sup_set)
        delta = round(percent * k)
        
        for j in range(delta):
            sup_set.append(np.random.randint(20000))
    return(p_indices)

In [ ]:
ex2 = deepcopy(ex1)

In [ ]:
ex2.p_indices = increase_support(ex2.p_indices, 1.5)

In [ ]:
ex2.update_cardinalities()

### Experiment 1. 02.03.20
#### Ledesgue constant on optimal designs w.r.t. weights

In [ ]:
points_test  = complex_area_pnts_gen(40000, ndim_input, distrib='LHS', mod = None)

In [ ]:
function = [None]

In [ ]:
ex1.error = ex1.lebesgue_all(points_test, function = function)[0]

In [ ]:
ex2.error = ex2.lebesgue_all(points_test, function = function)[0]

In [ ]:
ex3.error = ex3.lebesgue_all(points_test, function = function)[0]

In [ ]:
ex4.error = ex4.lebesgue_all(points_test, function = function)[0]

In [ ]:
## ex1 - optimal full sequence of designs
## ex_ext - expand experiment: k unique cardinalities form the set of expansions
def expand_preproc(ex_opt, ex_ext):
    print(ex_ext.cardinalities)
    print(ex_ext.expans)
    border_exp = ex_ext.expans[-1]
    k_up = np.where(ex_opt.cardinalities == border_exp)[0][-1]
    ex_opt.expans = ex_opt.expans[:k_up+1]
    ex_opt.p_indices = ex_opt.p_indices[:k_up + 1]
    ex_opt.update_cardinalities()
    return(ex_opt)
    

In [ ]:
ex1 = expand_preproc(ex1,ex4)

In [ ]:
fig = plt.figure()
plt.yscale('log')
plt.plot(ex1.error, 'ro', label = 'optimal designs')
#plt.plot(Leb_ex2[0], 'bo', label = 'expanded cut')
plt.plot(ex4.error, 'go',label = 'optimal extended with free poly to square matrix')
#plt.plot(Leb_ex4[0], 'yo', label = 'pts cutted to square')
plt.ylabel('Lebesgue constant, logscale', rotation=90, labelpad=5)
plt.xlabel('Number of expansion terms')
fnpdf = dir_points+'Lebesgue on changing designs.pdf'
plt.legend()
plt.grid(True)

plt.savefig(fnpdf)
plt.close(fig)

In [ ]:
### Experiment setup
error_name_set = ['random','lhs','sobol']
error_rand = 1
error_lhs = 1
error_sobol = 1
error_mask = np.array((error_rand, error_lhs, error_sobol))
error_tens_inx = np.where(error_mask == True)[0].astype(int)
error_set = [error_name_set[i] for i in error_tens_inx] 
solve_all_mesh = False
N_iter = 1
col_to_fix = []
point_to_fix = ["all"]
slice_coeff = None
exp_solve = [[],[],[]]
if solve_all_mesh:
    exp_solve = [np.unique(np.arange(N_col[0],N_col[-1])),[],None]
    exp_plot[0] = col_to_fix
    exp_plot[1] = point_to_fix
    exp_plot[2] = slice_coeff
else:
    exp_solve[0] = col_to_fix
    exp_solve[1] = point_to_fix
    exp_solve[2] = slice_coeff
    exp_plot = exp_solve

In [ ]:
mask = index_preprocess(p_indices,N_col,exp_solve)

In [ ]:
mesh

In [ ]:
## Error calculation
# function = [None]
mesh  = np.copy(mask)
# mesh[:,1] = Nwowge_col[mask[:,1]]
mesh[:,1] = ex4.expans[mask[:,1]]
mesh[:,0] = ex4.cardinalities
#for function in function_set:

In [ ]:
mesh1  = np.empty((len(ex1.p_indices), 2))
# mesh[:,1] = Nwowge_col[mask[:,1]]
mesh1[:,1] = ex1.expans
mesh1[:,0] = ex1.cardinalities

In [ ]:
#leb_1 = mult_error_tensor(N_iter, mesh, function, points_test, error_set, shape=None)

In [ ]:
k_d, k_up = ex4.expans[0]-3, ex4.expans[-1]-2
ex1.expans = ex1.expans[k_d:k_up]
ex1.p_indices = ex1.p_indices[k_d:k_up]
ex1.error = ex1.error[k_d:k_up]

In [ ]:
ex1.expans, ex1.cardinalities

In [ ]:
ex4.expans, ex4.cardinalities

In [ ]:
tensor_name = 'leb_tensor'
np.savez(os.path.join(dir_points, tensor_name), error_tensor=leb)

In [ ]:
if function == [None]:
    print('here')
    nme = tensor_name = 'error_leb_' + std_design_name + add_name4
    print(nme)
    tensor_name = nme
    error_saved_t = np.load(os.path.join(dir_points, tensor_name) + ".npz")
    error_tensor_r = error_saved_t['error_tensor']
else:
    
    tensor_name = tensor_name = 'error_{}_'.format(function[0].__name__) + std_design_name + add_name4
    error_saved_t = np.load(os.path.join(dir_points, tensor_name) + ".npz")
    error_tensor = error_saved_t['error_tensor']

In [ ]:
try:
    nme_add = nme + '_add'
    error_saved_add = np.load(os.path.join(dir_points, nme_add) + ".npz")
    error_tensor_add = error_saved_add['error_tensor']
    error_tensor_r = np.concatenate((error_tensor_r, error_tensor_add), axis = 2)
except:
    pass    

try:
    nme_add = nme + '_add2'
    error_saved_add = np.load(os.path.join(dir_points, nme_add) + ".npz")
    error_tensor_add = error_saved_add['error_tensor']
    error_tensor_r = np.concatenate((error_tensor_r, error_tensor_add), axis = 2)
except:
    pass   

try:
    nme_add = nme + '_add3'
    error_saved_add = np.load(os.path.join(dir_points, nme_add) + ".npz")
    error_tensor_add = error_saved_add['error_tensor']
    error_tensor_r = np.concatenate((error_tensor_r, error_tensor_add), axis = 2)
except:
    pass 

try:
    nme_add = nme + '_add4'
    error_saved_add = np.load(os.path.join(dir_points, nme_add) + ".npz")
    error_tensor_add = error_saved_add['error_tensor']
    error_tensor_r = np.concatenate((error_tensor_r, error_tensor_add), axis = 2)
except:
    pass   

try:
    nme_add = nme + '_add5'
    error_saved_add = np.load(os.path.join(dir_points, nme_add) + ".npz")
    error_tensor_add = error_saved_add['error_tensor']
    error_tensor_r = np.concatenate((error_tensor_r, error_tensor_add), axis = 2)
except:
    pass 

In [ ]:
error_tensor = error_tensor_r[error_tens_inx, :, :]
print(error_tensor.shape)

In [ ]:
def error_tensor_plot_expand(exp_list, mean_tensor, std, error_set, mesh, mesh_opt, dir_points, confidence=False):
    fig = plt.figure()
    plt.yscale('log')
    for k, points_type in enumerate(error_set):
        if confidence:
            plt.errorbar(mesh[:,0], mean_tensor[k], yerr=std[k],fmt='o', c='g', elinewidth=0.7,alpha=0.5, label='95% CI_' + points_type, ms=1.2)
            #c='tab:orange'   
        else:
            #plt.scatter(mesh[:,0], mean_tensor[k], s=0.7, label=points_type)
            plt.plot(mesh[:,0], mean_tensor[k], '--o', linewidth=0.7, label=points_type, markersize=1.2)
    
    
    plt.scatter(mesh[:,0], ex4.error, s=1.5, c='r', label=ex4.name)
    plt.scatter(mesh_opt[:,0], ex1.error, s=0.7, label=ex1.name)

    plt.xlabel(r'Number of points $k$ in designs $\xi_k$', fontsize=10)
    #plt.xlabel(r'Number of points in designs, $k$', fontsize=10)
    fnpdf = dir_points + 'err_leb_expand_n.pdf'
   
    # plt.ylabel('Approximation error, $\epsilon$', rotation=90, labelpad=5)
    plt.ylabel('Lebesgue constant', rotation=90, labelpad=5)
    #plt.legend()
    plt.legend(loc = 'upper center',bbox_to_anchor=(0.5, -0.15), ncol=2)
#     plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.tight_layout()

    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)

In [ ]:
def error_tensor_plot(exp_list, mean_tensor, T_up, T_down, error_set, inx, mask, experiment_params, dir_points,
                      confidence=False):
    ax = experiment_params[0]
    if experiment_params[0] == 2:
        ax = 0
    fig = plt.figure()
    plt.yscale('log')
    for k, points_type in enumerate(error_set):
        plt.plot(mask[:, ax][inx], mean_tensor[k][inx])#, label=points_type)
        if confidence:
            plt.fill_between(mask[:, ax][inx], T_down[k][inx], T_up[k][inx], alpha=0.4, label='95% CI_' + points_type)
    for i, obj in enumerate(exp_list):
        plt.scatter(mask[:, ax][inx], obj.error[inx], s=0.7, label=obj.name)

    if experiment_params[0] == 1:
        plt.xlabel('Number of basis functions', fontsize=10)
        fnpdf = dir_points + 'err(cols)_points={}_func={}.pdf'.format(experiment_params[1], experiment_params[2])
    elif experiment_params[0] == 0:
        plt.xlabel('Number of points', fontsize=10)
        fnpdf = dir_points + 'err(rows)_monoms={}_func={}.pdf'.format(experiment_params[1], experiment_params[2])
    else:
        plt.xlabel('Number of points', fontsize=10)
        fnpdf = dir_points + 'err(points)_coef={}_func={}.pdf'.format(experiment_params[1], experiment_params[2])
    # plt.ylabel('Approximation error, $\epsilon$', rotation=90, labelpad=5)
    plt.ylabel('Lebesgue constant', rotation=90, labelpad=5)
    plt.legend()
    #plt.legend(loc = 'upper center',bbox_to_anchor=(0.5, -0.17), ncol=2)
#     plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    #plt.tight_layout()

    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)

In [ ]:
### Plotting
function_set = function
exp_set = [ex1, ex2]
if function[0] == None:
    func_name = 'None'
else:
    func_name = function[0].__name__

print(error_tensor.shape)
z = 1.96   #z-value for 95% confidence interval
k = z / np.sqrt(error_tensor.shape[2])    #correcting by a square root from a number of experiments
mean_tensor = np.mean(error_tensor,axis = 2)
print(mean_tensor.shape)

std = np.std(error_tensor, axis = 2)
# std[0][4] *= 0.7
# std[0][6] *= 0.5
#std[1][0] *= 0.65
# std[0][-8] *= 0.7

T_up = mean_tensor + k * std
T_down = mean_tensor - k * std

for axis, jx in enumerate(exp_plot[:-1]):
    for count, fixed_entity in enumerate(jx):
        local_design = [[],[],None]
        local_design[axis] = [fixed_entity]
        print(local_design)
        #sub_mask = index_preprocess(p_indicesn,N_coln,local_design)
        #ix_internal = submask(mask, sub_mask)
        print('here')
        error_tensor_plot(exp_set,mean_tensor,T_up,T_down,error_set, np.arange(66) ,mesh,[axis, fixed_entity,func_name], dir_points, confidence = True)
        #error_tensor_plot_expand(exp_set, mean_tensor, k*std, error_set, mesh, mesh1, dir_points, confidence=False)
if exp_plot[-1]:
    local_design = [[],[],[]]
    local_design[2] = exp_plot[-1]
    sub_mask = index_preprocess(p_indicesn, N_coln,local_design)
    print(mask)
    print(sub_mask)
    ix_internal = submask(mask, sub_mask)
    print('internal',ix_internal)
    error_tensor_plot(exp_set,mean_tensor,T_up,T_down,error_set, ix_internal,mask,[2,exp_plot[-1],function.__name__], confidence = True)

In [ ]:
mean_tensor

In [ ]:
T_down[:,:5]

In [ ]:
std[:,:5]

In [ ]:
[(mask == sub_mask[i,:]).all(axis=1).nonzero()[0][0] for i in range(sub_mask.shape[0])]

In [ ]:
fig = plt.figure()
plt.yscale('log')
plt.plot(np.divide(mean_tensor[0,:], ex1.error), 'ro', label = 'optimal designs')
plt.ylabel('Lebesgue constant relation, logscale', rotation=90, labelpad=5)
plt.xlabel('Number of expansion terms')
fnpdf = dir_points+'nmb_vs_wmxv_pser.pdf'
plt.legend()
plt.grid(True)

plt.savefig(fnpdf)
plt.close(fig)

In [ ]:
plt.plot(np.array((1,2,3,1)), np.array((2,4,5,8)), 'o')

### Random points plot

### Draw areas

In [ ]:
#col_exp = min_expansion
col_exp = 40
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]
N_points = len(p_indices[i])

In [ ]:
parameter_plot(len(p_indices[i]))

In [ ]:
taken_points = np.load("piston_10_error.npz")

In [ ]:
x = taken_points['er']

In [ ]:
data = np.load("piston_10.npz")

In [ ]:
er_lhs = data['er_lhs']
er_r = data['er_r']
er_s = data['er_s']

### Radial functions

In [ ]:
def ellipse_sp(x,y):
    b,e = 0.2,0.95
    sigma,n = 1/2, 1
    r = sp.sqrt(x**2 + y**2)
    phi = sp.atan2(y,x)
    R = r*sp.sqrt(1 - (e*sp.cos(phi))**2)/b
    return((sp.exp(-1*(R**2)/(2*(sigma**2))))*sp.cos(n*phi))

def sin_blob_sp(x,y):
    a = 0.2
    b = 0.8
    sigma,n = 1,7
    r = sp.sqrt(x**2 + y**2)
    phi = sp.atan2(y,x)
    R = r*(1/(b + a*sp.cos(n*phi)))
    return((sp.exp(-1*(R**2)/(2*(sigma**2))))*(r**2))

f_ellipse = symb_to_func(ellipse_sp,    2, True, False, name='Ellipse')  
f_sin_blob= symb_to_func(sin_blob_sp,   2, True, False, name='Blob')  

f_ellipse.diff    = MakeDiffs(ellipse_sp,  2)
f_sin_blob.diff   = MakeDiffs(sin_blob_sp, 2)

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.gca(projection='3d')

# Make data.
a = 1.1
X = np.linspace(-a, a, 1000)
Y = np.linspace(-a, a, 1000)
phi = np.arange(0,2*np.pi, 0.01)
b= 0.2
e = 0.95
X, Y = np.meshgrid(X, Y)

#r = (b / np.sqrt(1-(e*np.cos(phi))**2))
#xx = r*np.cos(phi)
#yy = r*np.sin(phi)

r = 0.8 + 0.2*np.cos(7*phi)
xx = r*np.cos(phi)
yy = r*np.sin(phi)

#Z = f_ellipse(X,Y)
#Z = f_sin_blob(X,Y)
Z = f_gauss_doubl(X,Y) - 2

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.summer,
                       linewidth=0,label = 'Trigonometric')
#fc = ax.plot(xx, yy,'bo', zs=0, zdir='z', label='curve in (x,y)')
# Customize the z axis.
ax.set_zlim(-1, 1)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.4, aspect=5)
plt.grid(True)
#fnpdf = 'ellips.pdf'
#plt.savefig(fnpdf)
plt.show()

In [ ]:
radiuses = ['0.0','0.01','0.015','0.02','0.025','0.03','0.04','0.05','0.06','0.07','0.08','0.09','0.1','0.11','0.12','0.13','0.15','0.18','0.2','0.25']#,'0.4']
funcs = [f_gauss,f_rosenbrock,f_sincos,f_schafferf6,f_roots,f_yaf1,f_gabor]

In [ ]:
for function in funcs:
    error_rad = []
    #volume = []
    for rad in radiuses:
        fn = dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt"
        N_row, N_col, p_indices = file_extraction(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt")
        ix = index_preproc(N_row,N_col,exp_solve,step_along=1, new_extr_path = None)

        #pts = p_indices[ix][0]

        #A = GenMat(N_col[ix][0]*(nder+1), x[pts], poly=cheb, debug=False, pow_p=1)
        #A = matrix_prep(A, nder+1)
        #vol = la.det(A.T @ A)
        #volume.append(vol)
        bmxvol_error = bmaxvol_error(function,points_test,p_indices,ix)
        error_rad.append(bmxvol_error)
    fig, ax = plt.subplots(1, 1, figsize=(12, 14))

    ax.set_xlim(-0.01, 0.26)
    plt.yscale('log')
    plt.xticks(np.array(radiuses,dtype=float), fontsize=5)
    plt.plot(np.array(radiuses,dtype=float),np.array(error_rad)[:,0,0])
    plt.xlabel('Erasion radius', fontsize=10)
    plt.ylabel('$\epsilon$')
    plt.grid(True)
    fnpdf = 'error-rad_fine_func={}.pdf'.format(function.__name__)
    plt.savefig(fnpdf)

### Weighted maxvol

In [ ]:
nder = 1
domain_type = None
p_size = 4
k = 1
n_pts = 8000
poly_type = poly_power
x = complex_area_pnts_gen(n_pts, nder, distrib='lhs', mod = domain_type)*k
x_max, x_min = np.round(np.amax(x)),np.floor(np.amin(x))
x_bounds = np.array([[-1*k],
                      [k]])
x = np.vstack((x, x_bounds))

In [ ]:
A = GenMat(p_size, x, poly=poly_type, debug=False, pow_p=1,ToGenDiff=True)
A = matrix_prep(A, nder+1)
#piv = rect_block_maxvol(A, nder, 18, max_iters=200, rect_tol = 0.05, tol = 0.0,debug = False, to_erase = None)
perm, C = block_maxvol(A, nder, tol = 0.05, max_iters=200, swm_upd=True)

In [ ]:
def rect_block_backward_core(C, P, nder, Kmax, t = 0.05, to_erase=None):
    ndim = nder + 1
    n, m = C.shape
    num_block = n // ndim
    k = int(Kmax // ndim)
    Fl = True
    block_index = m // ndim
    non_unique_block_piv = np.copy(P[:m:ndim]//ndim)
    block_indcs = P[::ndim]//ndim
    S = cold_start_tens(C,ndim)

    while Fl and block_index < k:
        det_list = la.det(np.eye(ndim) + S)
        elem = np.argmax(det_list)
        
        if det_list[elem] > (1 + t):
            range_new_block = np.arange(elem*ndim, elem*ndim + ndim)

            
            #------ update part -----            
            block = np.eye(ndim) + C[range_new_block].dot(C[range_new_block].T)
            op3 = C.dot(la.solve(block,C[range_new_block]).T)
            op4 = np.dot(op3, C[range_new_block])
                   
            
            C = np.hstack((C - op4, op3))
            S = cold_start_tens(C,ndim)
        
            if not np.isin(block_indcs[elem],non_unique_block_piv):
                block_index += 1
            
            non_unique_block_piv = np.hstack((non_unique_block_piv,block_indcs[elem]))
        else:
            print('No relevant elements found')
            Fl = False
    print(block_index)
    return(non_unique_block_piv, np.unique(non_unique_block_piv))


def brect_backward_naive(A,nder,Kmax,t=0.05):
    ndim = nder + 1
    n, m = A.shape
    k = int(Kmax // ndim)
    ids = np.copy(A[:m])
    C = np.dot(A,np.linalg.pinv(ids))
    
    block_index = m // ndim
    
    block_indcs = np.arange(int(n//ndim))
    non_unique_block_piv = np.copy(block_indcs[:(int(m//ndim))])
    S = cold_start_tens(C,ndim)
    
    Fl = True
    
    test_iter = 0
    
    while Fl and block_index < k and test_iter < 150:
        det_list = la.det(np.eye(ndim) + S)
        elem = np.argmax(det_list)
        if det_list[elem] > (1 + t):
            range_new_block = np.arange(elem*ndim, elem*ndim + ndim)
            ids = np.vstack((ids,A[range_new_block]))
            
            if not np.isin(block_indcs[elem],non_unique_block_piv):
                block_index += 1
            
            non_unique_block_piv = np.hstack((non_unique_block_piv,block_indcs[elem]))
            
            C = np.dot(A,np.linalg.pinv(ids))
            S = cold_start_tens(C,ndim)
            test_iter += 1
        else:
            print('no elements found')
            Fl = False
    
    
    return(non_unique_block_piv, np.unique(non_unique_block_piv))

def rect_backward_naive(A,Kmax,t=0.01):
    def cold_start_S(C):
        S = np.empty(C.shape[0])
        for i in range(C.shape[0]):
            S[i] = C[i] @ C[i].T
        return(S)
    ndim = 1
    n, m = A.shape
    ids = np.copy(A[:m])
    C = np.dot(A,np.linalg.pinv(ids))
    S = cold_start_S(C)
    P = np.arange(n)
    non_unique_piv = np.copy(P[:m])
    row_index = m
    Fl = True
    test_iter = 0
    while Fl and row_index < Kmax and test_iter < 1000:
       
        elem = np.argmax(S)
        #print(S[elem])
        if S[elem] > (t):
            print(test_iter)
            ids = np.vstack((ids,A[elem]))
            #print(ids)
            if not np.isin(P[elem],non_unique_piv):
                row_index += 1
            
            non_unique_piv = np.hstack((non_unique_piv,P[elem]))
            #print(non_unique_piv)
            
            C = np.dot(A,np.linalg.pinv(ids))
            S = cold_start_S(C)
            test_iter += 1
        else:
            print('no elements found')
            Fl = False
    
    
    return(non_unique_piv)
    

In [ ]:
a,b = maxvol(A)

In [ ]:
idx_n,idx_o = change_intersept(np.arange(p_size),a)

In [ ]:
A[idx_n,:] = A[idx_o,:]
x[idx_n] = x[idx_o]

In [ ]:
nup = rect_backward_naive(A,90,t=0.001)

In [ ]:
rr,tt = rect_block_maxvol_core(C, perm, nder, 60)

In [ ]:
print(la.det(GenMat(p_size, x[:4], poly=poly_type, debug=False, pow_p=1,ToGenDiff=False)))
print(la.det(GenMat(p_size, x_bas, poly=poly_type, debug=False, pow_p=1,ToGenDiff=False)))

In [ ]:
pts,weight = np.unique(nup,return_counts=True)

In [ ]:
pts

In [ ]:
weight

In [ ]:
def plot_chosen_points(x,pts,weight):
    
    if x.shape[1] == 1:
        plt.figure(figsize=(20,8))
        for i,wt in enumerate(weight):
            plt.plot(x[pts[i]],1,'bo',mew=0.5+np.log(wt), ms=5)
    elif x.shape[1] == 2:
        plt.figure(figsize=(10,10))
        for i,wt in enumerate(weight):
            plt.plot(x[pts[i],0],x[pts[i],1],'bo',mew=0.5+np.log(wt), ms=5)
    plt.show()
        

In [ ]:
plot_chosen_points(x,pts,weight)

In [ ]:
basis_indc = [pt for i, pt in enumerate(pts) if weight[i]>80]
basis_indc

In [ ]:
x_bas = np.copy(x[basis_indc])

In [ ]:
x_bas

In [ ]:
def uwu(x, A,p_size, poly_type):
    c = np.empty(x.shape[0])
    for cnt in range(x.shape[0]):
        func_vec = GenMat(p_size, np.array([[x[cnt]],]), poly=poly_type, debug=False, pow_p=1,ToGenDiff=False)
        c[cnt] = np.dot(func_vec, np.dot(A,func_vec.T))
    return(c)

In [ ]:
def doom_precise(x, x_bas, n, x_min, x_max):
    k = np.abs(x_max - x_min)*0.01*0.5
    
    for i in range(x_bas.shape[0]):
        if x_bas[i] > x_min and x_bas[i] < x_max :
            precise = np.linspace(x_bas[i]-k, x_bas[i]+k, n)
            precise = np.reshape(precise,(n,1))
            x = np.vstack((x,precise))
    return(x)

In [ ]:
### Domain must include x_bas in the beginning
def accurate_debug(p_size, domain, x_bas, n_iter, poly_type, mod = 'argmax'):
    def cold_start_S(C):
        S = np.empty(C.shape[0])
        for i in range(C.shape[0]):
            S[i] = C[i] @ C[i].T
        return(S)
    A_bas = GenMat(p_size, x_bas, poly=poly_type, debug=False, pow_p=1,ToGenDiff=False)
    A = GenMat(p_size, domain, poly=poly_type, debug=False, pow_p=1,ToGenDiff=False)
    n,m = A.shape
    C = np.dot(A,np.linalg.pinv(A_bas))
    S = cold_start_S(C)
    P = np.arange(n)
    non_unique_piv = np.copy(P[:x_bas.shape[0]])
    row_index = x_bas.shape[0]
        
    QF_tensor = np.empty((p_size, p_size, n_iter))
    c_tensor = np.empty((n, n_iter))
    
    for i in range(n_iter):
        A_bas_inv = la.pinv(A_bas)
        QF = (A_bas_inv) @ A_bas_inv.T    
        QF_tensor[:,:,i] = QF
        c_tensor[:,i] = S 
        if mod == 'argmax':
            elem = np.argmax(S)
        else:
            elem = np.argmin(S)
        #print(i)
        A_bas = np.vstack((A_bas,A[elem]))
        if not np.isin(P[elem],non_unique_piv):
            row_index += 1

        non_unique_piv = np.hstack((non_unique_piv,P[elem]))
        #print(non_unique_piv)

        C = np.dot(A,np.linalg.pinv(A_bas))
        S = cold_start_S(C)
    
    
    return(QF_tensor, c_tensor, non_unique_piv)


def block_debug(p_size,domain,x_bas,nder,n_iter, poly_type):
    ndim = nder + 1
    A_bas = GenMat(p_size, x_bas, poly=poly_type, debug=False, pow_p=1,ToGenDiff=True)
    A = GenMat(p_size, domain, poly=poly_type, debug=False, pow_p=1,ToGenDiff=True)
    
    A = matrix_prep(A, nder+1)
    A_bas = matrix_prep(A_bas, nder+1)
    n, m = A.shape

  
    C = np.dot(A,np.linalg.pinv(A_bas))
    
    block_index = m // ndim
    
    block_indcs = np.arange(int(n//ndim))
    non_unique_block_piv = np.copy(block_indcs[:(int(m//ndim))])
    S = cold_start_tens(C,ndim)
    
    QF_tensor = np.empty((p_size, p_size, n_iter))
   
    for i in range(n_iter):
        A_bas_inv = la.pinv(A_bas)
        QF = (A_bas_inv) @ A_bas_inv.T    
        QF_tensor[:,:,i] = QF
        det_list = la.det(np.eye(ndim) + S)
        elem = np.argmax(det_list)
        
        range_new_block = np.arange(elem*ndim, elem*ndim + ndim)
        A_bas = np.vstack((A_bas,A[range_new_block]))


        non_unique_block_piv = np.hstack((non_unique_block_piv,block_indcs[elem]))

        C = np.dot(A,np.linalg.pinv(ids))
        S = cold_start_tens(C,ndim)
        
    return(QF_tensor, non_unique_block_piv)

def symbolic_maxvol(p_size,poly_type, x_bas, n_iter, x_min, x_max):
    QF_tensor = np.empty((p_size, p_size, n_iter))
    xb = np.copy(x_bas)
    x_add = []    
    x = Symbol('x')
    func_list = [x**i for i in range(p_size)]
    func_vec = np.array(func_list)
    bin_point = np.array([[-20],
                           [80]])
    for i in range(n_iter):
        A_bas = GenMat(p_size, xb, poly=poly_type, debug=False, pow_p=1,ToGenDiff=False)
        A_bas_inv = la.pinv(A_bas)
        QF = (A_bas_inv) @ A_bas_inv.T    
        QF_tensor[:,:,i] = QF
        summand = simplify(np.dot(func_vec, QF.dot(func_vec.T)))
        
        maxs = solve(diff(summand,x,1), x)
        maxs.append(x_min)
        maxs.append(x_max)
        ind = []
        for j, maxj in enumerate(maxs):
            
            if isinstance(maxj, Add):
                ind.append(j)
        
        maxs = [y for z, y in enumerate(maxs) if z not in ind]
        
        maxs = np.array(maxs, dtype = float)
        uwu = np.argmax(fx(QF, maxs))
        #p = np.random.randint(2)
        x_add.append(maxs[uwu])
        #modular = [bin_point[p],x_min,x_max]
        #modular = np.array(modular,dtype=float)
        #uwu = np.argmax(fx(QF, modular))
        xb = np.vstack((xb,maxs[uwu]))
        #xb = np.vstack((xb,modular[uwu]))
    return(x_add,QF_tensor)       

### Aim: 
#### To see difference in greedy expansion algorithm
Run greedy expansion with allowed repetitions / difference is in initial set.
May be we can refuse square maxvol in a Vandermonde matrices ?

In [ ]:
def basis_generator(x, x_bas, ci, shape):
    x_bas = np.sort(x_bas, axis = 0)
    if ci >= 0.5 :
        mask = np.where(np.abs(x - x_bas) < (1-ci))[0]
    else:
        x_outlyer = np.empty(x_bas.shape[0] - 1)
        for i, x_b in enumerate(x_bas[:-1]):
            x_outlyer[i] = x_b +  np.abs(x_bas[i+1] - x_b)/2
        mask = np.where(np.abs(x - x_outlyer) < ci)[0]#[0] if i==0 else np.vstack((np.where(x - x_out < ci)[0], mask))
    nodes = np.random.choice(mask, shape)
    return(nodes)

### Experiment log entry /1

#### basis: 4 poly
#### 4 doomed initial

In [ ]:
idx_n,idx_o = change_intersept(np.arange(p_size),basis_indc)
x[idx_n] = x[idx_o]

In [ ]:
x_bas = np.copy(x[:4])

In [ ]:
x_bas[0] = 20
x_bas[1] =  80
x_bas[3] =  -80
x_bas[2] = -20

In [ ]:
x_bas

In [ ]:
ke,QFtS = symbolic_maxvol(p_size,poly_type,x_bas,200,x_min,x_max)

In [ ]:
#tt = [5,3,1,5]
del tt[0]
print(tt)

In [ ]:
QFt, ct, piv = accurate_debug(p_size, x, x_bas, 500,poly_type, mod = 'argmax')
#QFt_min, _, piv_min = accurate_debug(p_size, x, x_bas, 300,poly_type, mod = 'argmin')

In [ ]:
ke

In [ ]:
pts_1,weight_1 = np.unique(piv,return_counts=True)

In [ ]:
print(x[pts_1])
print(weight_1)

In [ ]:
plot_chosen_points(x,pts_1,weight_1)

In [ ]:
fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
t = np.linspace(x_min, x_max, 1000)


state_num = QFt.shape[2]




init_state = 7
l, = plt.plot(t,uwu(t,QFt[:,:,init_state],p_size, poly_type),'bo')
plt.axis([x_min, x_max, 0, 1])

axcolor = 'lightgoldenrodyellow'
axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)


samp = Slider(axamp, 'Evolut', 0, state_num-1, valinit=init_state, valstep=int(1))

def update(val):
    amp = int(samp.val)
    #l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    l.set_ydata(uwu(t,QFt[:,:,amp],p_size, poly_type))
    
    ax.axis([x_min,x_max,0,1.2*np.amax(uwu(t,QFt[:,:,amp],p_size, poly_type))])
    fig.canvas.draw_idle()

samp.on_changed(update)

plt.show()

### Experiment log entry /3

#### basis: 3 poly
#### 3 random initial, doomed are hyding in the set

### Experiment log entry /4

#### basis: 4 poly
#### Initial set: overdetermined basis FAR FAR away from doomed points
Ok, seems that points converge to the doomed ones, and local optima are not always in the doomed points in the beggining of iteration process. Adding points changes matrix of the quadratic form, and therefore changes coefficients of the polynom. But they somehow converge...

In [ ]:
x_bas

In [ ]:
x_bas_2

In [ ]:
idx = basis_generator(x, x_bas, ci = 0.1,shape = 4)

In [ ]:
x_bas_x = np.array([[-0.85],
                   [0.84],
                   [0.778],
                   [0.94],
                   [-0.89],
                   [0.84],
                   [0.778],
                   [0.94]])

In [ ]:
idx_n,idx_o = change_intersept(np.arange(idx.shape[0]),idx)
x[idx_n] = x[idx_o]

In [ ]:
x_bas_overdet = x[:idx.shape[0]]

In [ ]:
x = np.vstack((x_bas_x,x))

In [ ]:
QFt_4, ct_4, piv_4 = accurate_debug(p_size, x, x_bas_x, 200,poly_type,mod='argmax')

In [ ]:
pts_4,weight_4 = np.unique(piv_4,return_counts=True)

In [ ]:
print(pts_4)
print(weight_4)

In [ ]:
mask = np.where(weight_4>20)

In [ ]:
print(x[pts_4[mask]])

In [ ]:
int(7/2)

In [ ]:
### evaluating test
for inital_points_distrib in inital_points_distribs:
    points_fn = 'taken_points_{}'.format(inital_points_distrib)
    x = complex_area_pnts_gen(num_points_for_big_matrix, nder, distrib='lhs', mod = domain_type)

    A = GenMat(p_size, x, poly=poly, debug=False, pow_p=1)
    A = matrix_prep(A, nder+1)
    
    np.savez(os.path.join(dir_str, points_fn), x=x, points_test=points_test)
    
    
    
    fn_pre_pdf = "distrib={}".format(inital_points_distrib)
    for cut_radius in cut_radiuses:
        f = open(os.path.join(dir_str, "distrib={}_radius={}".format(inital_points_distrib, cut_radius) + '.txt'), "w")
        for expansion in range(min_expansion, max_expansion):
                    for N_rows_ex in range(max_row, expansion, -1): # It's not the way people do...
                        N_rows = N_rows_ex*(nder+1)
                        fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(expansion, N_rows_ex))
                        try:
                            taken_points = test_bm(A, x,nder, expansion, N_rows, cut_radius = cut_radius,to_save_pivs=N_rows_ex==max_row, 
                                                       fnpdf=fnpdf)
                        except SingularError as err:
                            print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                                                expansion, N_rows_ex, err.value)) 
                            #continue
                            break
                            


                        taken_points.tofile(f, sep=" ")
                        f.write("_Nrows={}_expans={}\n".format(N_rows, expansion))
                        f.flush()

        f.close()
